In [13]:
import requests
import pandas as pd

In [14]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0",
    "Accept-Language": 'en-US,en;q=0.9',
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Referer": "https://shopee.vn/",
    "From": "",
    "af-ac-enc-dat": "",
    "x-api-source": "pc"
}

### EXTRACT CATEGORY

In [15]:
URL = "https://shopee.vn/api/v4/official_shop/get_categories"
PARAMS = { 
}

In [16]:
response = requests.get(URL, headers=HEADERS, params=PARAMS)
if response.status_code == 200:
    data = response.json()["data"]["categories"]
    category_data = []

    for category in data:
        category_id = category["category_id"]
        name = category["name"]
        image = category["image"]

        category_data.append({
            "category_id": category_id,
            "name": name,
            "image": image
        })
    category = pd.DataFrame(category_data)

else :
    print(f"Error: {response.status_code}")

In [17]:
category.head()

,category_id,name,image
0,-1,All,None
1,11035478,Thể Thao & Du Lịch,6cb7e633f8b63757463b676bd19a50e4
2,11036793,Ô tô - xe máy - xe đạp,3fb459e3449905545701b418e8220334
3,11036525,Bách hóa Online,c432168ee788f903f1ea024487f2c889
4,11036670,Nhà Cửa & Đời Sống,24b194a695ea59d384768b7b471d563f


### TRANSFORM CATEGORY

In [18]:
# remove -1	All	None
category = category[category["category_id"] != -1]
print(f"Total category fetched: {len(category)}")

Total category fetched: 25


### EXTRACT SHOP

In [19]:
categories = category['category_id'].tolist()  # Convert the 'category_id' column to a list

In [20]:
shop_data = []

for cat_id in categories:
    URL = f"https://shopee.vn/api/v4/official_shop/get_shops_by_category?need_zhuyin=0&category_id={cat_id}"
    PARAMS = {}

    response = requests.get(URL, headers=HEADERS)
    if response.status_code == 200:
        data = response.json()
        cat_id_res = data["data"]["cat_id"]  # Get the category_id from the response

        brands = data["data"]["brands"]

        for brand in brands:
            brand_ids = brand["brand_ids"]
            for shop in brand_ids:
                username = shop.get("username", "")
                brand_name = shop.get("brand_name", "")
                shopid = shop.get("shopid", "")
                logo = shop.get("logo", "")
                logo_pc = shop.get("logo_pc", "")
                shop_collection_id = shop.get("shop_collection_id", "")
                ctime = shop.get("ctime", "")
                brand_label = shop.get("brand_label", "")
                shop_data.append({
                    "category_id": cat_id_res,
                    "username": username,
                    "brand_name": brand_name,
                    "shop_id": shopid,
                    "logo": logo,
                    "logo_pc": logo_pc,
                    "shop_collection_id": shop_collection_id,
                    "ctime": ctime,
                    "brand_label": brand_label
                })
    else:
        print(f"Error: {response.status_code}")

shop = pd.DataFrame(shop_data)
print(f"Total shop fetched: {len(shop)}")

Total shop fetched: 3583


In [21]:
# Shop for a specific category

# URL = "https://shopee.vn/api/v4/official_shop/get_shops_by_category?need_zhuyin=0&category_id=11035478"
# PARAMS = {
# }

# response = requests.get(URL, headers=HEADERS)
# if response.status_code == 200:
#     data = response.json()
#     brands = data["data"]["brands"]
#     shop_data = []
    
#     for brand in brands:
#         brand_ids = brand["brand_ids"]
#         for shop in brand_ids:
#             username = shop["username"]
#             brand_name = shop["brand_name"]
#             shopid = shop["shopid"]
#             logo = shop["logo"]
#             logo_pc = shop["logo_pc"]
#             shop_collection_id = shop["shop_collection_id"]
#             ctime = shop["ctime"]
#             brand_label = shop["brand_label"]

#             shop_data.append({
#                 "username": username, 
#                 "brand_name": brand_name, 
#                 "shopid": shopid, 
#                 "logo": logo, 
#                 "logo_pc": logo_pc, 
#                 "shop_collection_id": shop_collection_id, 
#                 "ctime": ctime, 
#                 "brand_label": brand_label
#             })
#     shop = pd.DataFrame(shop_data)
#     print(f"Total shop fetched: {len(shop)}")

# else:
#     print(f"Error: {response.status_code}")

In [22]:
shop

,category_id,username,brand_name,shop_id,logo,logo_pc,shop_collection_id,ctime,brand_label
0,11035478,adidasofficialstore,Adidas Official Store,71009635,fb1da7967deaf566624ebc9a4d326272,3a7da458d20218cfb0db50b7c6377812,0,1620981895,0
1,11035478,akisport.net,Aki Sport,432189904,69be26b09d0533ee67cc18f0566d6515,eea895f6e058cbef2755b7a133d084fc,0,1630297767,0
2,11035478,amalife_vietnam,Amalife Việt Nam,551582212,8b8dd325828b99cb15d1c13a5662fdf0,b8ee116e29e5eec858be6a8e50fece5c,0,1636012307,0
3,11035478,_andrealin,Andrealin,388588353,vn-50009109-436671da9ef02e74801cee020093837c,vn-50009109-ce8ff3b80a6a6f80a065742c620ddd04,0,1687249543,0
4,11035478,anta.official,Anta Flagship Store,93177350,f4fa3803898c22e95a1fe6515c96c1fc,7af9076c414c6e2495bb56649493835c,0,1620981895,0
...,...,...,...,...,...,...,...,...,...
3578,11036345,yuwellhanoiofficialstore,yuwellhanoi82,767291332,e071aaeaa9458663f09725e8a0cc2049,2ee4efd37941a5f10daa1b485f36edf8,0,1657597160,0
3579,11036345,ziaja.baniphar,Ziaja Baniphar Official,408810138,73e792f890cc3603f9301ee9606afcd1,db17417b10d68ab74c07bec1f9bda118,0,1637654429,0
3580,11036345,1sk_official,1SK Official Shop,872131820,vn-50009109-ef32974b88cd0327362ba2a69f3a189c,vn-50009109-a826caef84ad390be3d3e2f26cc6c9f7,0,1681439584,0
3581,11036345,3ce.official.vn,3CE,285380605,26d2bde1bb7c53f8075be5cbbfa65040,db17a21b95d731befb656fb9167c301f,0,1631709921,0
